<a href="https://colab.research.google.com/github/Mushkaan/Study-area-New-Delhi/blob/main/bigdata_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as map
import ee
ee.Authenticate()
ee.Initialize(project='ee-mushkaan2024')
m = map.Map()

In [2]:
Gujarat = ee.FeatureCollection('projects/ee-mushkaan2024/assets/Gujarat')
m.addLayer(Gujarat, {}, 'Gujarat')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
vis_para = {
    'min': 0,
    'max': 100,
    'palette': ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']}

In [4]:
legend = {'Treecover': '#006400',
          'Shrubland': '#ffbb22',
          'Grassland': '#ffff4c',
          'Croplands': '#f096ff',
          'Built-up': '#fa0000',
          'Bare/Sparse vegetation': '#b4b4b4',
          'Snow-ice': '#f0f0f0',
          'Permanent water bodies': '#0064c8',
          'Herbaceous wetland': '#0096a0',
          'Mangroves': '#00cf75',
          'Moss and Lichen': '#fae6a0'
          }

In [10]:
gujarat_esa = ee.ImageCollection('ESA/WorldCover/v200').mean().clip(Gujarat).select('Map')
m.addLayer(gujarat_esa, vis_para, 'ESA WorldCover');
m.add_legend(Title = "Legend", legend_dict=legend,**vis_para)

HTML(value="<html>\n<body>\n  <div class='my-legend'>\n  <div class='legend-title'>Legend</div>\n  <div class=…

In [11]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterBounds(Gujarat).filterDate('2022-01-01','2023-01-01').filterMetadata('CLOUD_COVER', 'less_than', 5).mean().clip(Gujarat)

In [7]:
TrainingP = gujarat_esa.sample(**{
    'region': Gujarat,
    'scale': 20,
    'numPixels': 4000,
    'seed': 1,
    'geometries': True
})
m.addLayer(TrainingP, {}, 'Training Points')

In [8]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6']
label = 'Map'
training = landsat8.select(bands).sampleRegions(**{
    'collection': TrainingP,
    'properties': [label],
    'scale': 20
})
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)
result = landsat8.select(bands).classify(trained)

In [ ]:
m.addLayer(result, vis_para, 'Supervised Classification')
m.addLayerControl(position = 'bottomleft')
m